In [ ]:
# Radar chart URL - https://python-graph-gallery.com/390-basic-radar-chart/
# list of references and ideas - https://towardsdatascience.com/jupyter-superpower-interactive-visualization-combo-with-python-ffc0adb37b7b
# https://observablehq.com/@rreusser/domain-coloring-for-complex-functions?collection=@observablehq/webgl
# http://users.mai.liu.se/hanlu09/complex/domain_coloring.html
# more radar https://python-graph-gallery.com/390-basic-radar-chart/
# more radar https://matplotlib.org/examples/api/radar_chart.html

In [ ]:
# Function to retrieve a secret value from AWS secrets manager. The secret name must be passed into the function as a parameter. The code has been adapted into a parameterized function from the canned template provided by AWS.
def get_secret(secret_name):

    #secret_name = "AmazonSageMaker-gmaps"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return secret 

In [ ]:
# Dependency to have 'pip install geoip' installed from configuration notebook.

import geoip2.database
reader = geoip2.database.Reader('tools/GeoLite2-City.mmdb')

#create a function to find the lat/long of the addy based on maxmind reader
def lookup_location(IPAddress):
    try:
        response = reader.city(IPAddress)
        return round(response.location.latitude,3), round(response.location.longitude,3), response.country.name, response.city.name
    except:
        # If private IP address, return address at center of map
        return 0,0,0,0
        #return null

#function to get a dual output into different columns in a dataframe
#https://stackoverflow.com/questions/23690284/pandas-apply-function-that-returns-multiple-values-to-rows-in-pandas-dataframe
def apply_and_concat(dataframe, field, func, column_names):
    return pd.concat((
                         dataframe,
                         dataframe[field].apply(
                         lambda cell: pd.Series(func(cell), index=column_names))), axis=1)

def get_location(df_ip,columnIP):
    df_min = apply_and_concat(df_ip, columnIP, lookup_location, ['latitude', 'longitude', 'country', 'state'])
    # Remove the rows with missing lat/long
    df_min = df_min[(df_min.latitude != 0.000)&(df_min.longitude != 0.000)]
    return df_min

In [ ]:
# Prepare GoogleMaps API integration

# pip install gmaps
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
import gmaps
import gmaps.datasets
import boto3
import json
import base64
from botocore.exceptions import ClientError

# Enter your Google Maps API key. You could use pickle to serialize it and store it outside of code.
# When retrieving a secret from SecretsManager, the default policy requires the secret to begin with AmazonSageMaker- in order to have access. If you are returning a value of None, it is likely there is a permissions issue.

def prepare_location(df_min):
    #pandas chaining
    #1 - group by lat/long
    #2 - get the size of a column to consolidate groupby object
    #3 - take it from a indexed series to a dataframe
    #4 - reset the index so that groupby is flattened and can be referenced
    #5 - rename the size column (0) to 'count'
    df_plot = df_min.groupby(['latitude', 'longitude']).size().to_frame().reset_index().rename(columns={0:'count'})
    return df_plot

# Need to enable extensions and widgets before figure will display
#jupyter nbextension enable --py gmaps
#jupyter nbextension enable --py widgetsnbextension
#Restart Jupyter

def get_heatmap(df_plot):
    locations = df_plot[['latitude', 'longitude']]
    weights = df_plot['count']
    secretname = 'AmazonSageMaker-gmaps'
    api = json.loads(get_secret(secretname))
    apisecret = api['gmapsapi']
    gmaps.configure(api_key=apisecret)
    fig = gmaps.figure()
    #fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
    fig.add_layer(gmaps.heatmap_layer(locations))
    return fig